In [1]:
import os
import numpy as np
from tf_keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tf_keras.models import Model
from tf_keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D


In [2]:

# Путь к директории с изображениями
data_dir = 'C:/Users/kimsl/VKR/1/dataset_prepro_autoencoder'

# Параметры данных
img_width, img_height = 600, 600
batch_size = 32

# Функция для загрузки и предобработки изображений
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=(img_width, img_height))
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)

# Загрузка и нормализация данных
images = load_images_from_folder(data_dir)
images = images.astype('float32') / 255.0


In [3]:

# Создание автоэнкодера
input_img = Input(shape=(img_width, img_height, 3))

# Энкодер
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Декодер
x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')


In [4]:
# Обучение модели
autoencoder.fit(images, images,
                epochs=50,
                batch_size=batch_size,
                shuffle=True)

# Сохранение модели
autoencoder.save('autoencoder_model.h5')

Epoch 1/50

24/63 [==========>...................] - ETA: 14:52 - loss: 0.0494

KeyboardInterrupt: 

In [6]:
import tensorflow as tf

# Проверка доступности GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


True